In [1]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 SERIAL RUNNER SCANNER")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("Mission: Find stocks that run repeatedly on cycles")
print("="*60)

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 SERIAL RUNNER SCANNER
Date: 2026-01-06 22:57
Mission: Find stocks that run repeatedly on cycles


In [2]:
# Our universe - 30 tickers across sectors we track
UNIVERSE = {
    'Space': ['SIDU', 'ASTS', 'LUNR', 'RKLB', 'RDW'],
    'Quantum': ['IONQ', 'QBTS', 'RGTI', 'QUBT'],
    'Semiconductor': ['NVTS', 'WOLF', 'ON', 'AEHR', 'SKYT', 'AEVA'],
    'Nuclear': ['UUUU', 'LEU', 'CCJ', 'SMR', 'OKLO'],
    'Rare_Earth': ['USAR', 'MP'],
    'AI_Infra': ['MU', 'WDC', 'STX', 'LITE']
}

ALL_TICKERS = [ticker for tickers in UNIVERSE.values() for ticker in tickers]
print(f"Scanning {len(ALL_TICKERS)} tickers across {len(UNIVERSE)} sectors")
print(f"Tickers: {ALL_TICKERS}")

Scanning 26 tickers across 6 sectors
Tickers: ['SIDU', 'ASTS', 'LUNR', 'RKLB', 'RDW', 'IONQ', 'QBTS', 'RGTI', 'QUBT', 'NVTS', 'WOLF', 'ON', 'AEHR', 'SKYT', 'AEVA', 'UUUU', 'LEU', 'CCJ', 'SMR', 'OKLO', 'USAR', 'MP', 'MU', 'WDC', 'STX', 'LITE']


In [3]:
def find_runs(ticker, min_run_days=3, min_gain_pct=10, lookback_months=6):
    """
    Find all momentum runs (consecutive green days) for a ticker.
    
    A 'run' is defined as:
    - 3+ consecutive green days (close > previous close)
    - Total gain > 10%
    
    Returns list of runs with:
    - Start date, end date
    - Length (days)
    - Total gain (%)
    - Days since last run
    """
    try:
        # Get historical data
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months * 30)
        
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if len(hist) < 20:
            return None, "Insufficient data"
        
        # Calculate daily returns
        hist['green'] = hist['Close'] > hist['Close'].shift(1)
        hist['pct_change'] = hist['Close'].pct_change() * 100
        
        runs = []
        current_run_start = None
        current_run_days = 0
        current_run_gain = 0
        
        for i, (idx, row) in enumerate(hist.iterrows()):
            if row['green']:
                if current_run_start is None:
                    current_run_start = idx
                    run_start_price = hist.loc[idx, 'Close'] / (1 + row['pct_change']/100)
                current_run_days += 1
            else:
                # Run ended - check if it qualifies
                if current_run_days >= min_run_days:
                    # Calculate total gain
                    run_end_idx = hist.index[i-1] if i > 0 else idx
                    run_end_price = hist.loc[run_end_idx, 'Close']
                    total_gain = ((run_end_price / run_start_price) - 1) * 100
                    
                    if total_gain >= min_gain_pct:
                        runs.append({
                            'start_date': current_run_start,
                            'end_date': run_end_idx,
                            'length': current_run_days,
                            'gain': total_gain,
                            'start_price': run_start_price,
                            'end_price': run_end_price
                        })
                
                # Reset
                current_run_start = None
                current_run_days = 0
        
        # Check if currently in a run
        if current_run_days >= min_run_days:
            run_end_idx = hist.index[-1]
            run_end_price = hist.loc[run_end_idx, 'Close']
            total_gain = ((run_end_price / run_start_price) - 1) * 100
            if total_gain >= min_gain_pct:
                runs.append({
                    'start_date': current_run_start,
                    'end_date': run_end_idx,
                    'length': current_run_days,
                    'gain': total_gain,
                    'start_price': run_start_price,
                    'end_price': run_end_price,
                    'active': True
                })
        
        # Calculate days between runs
        for i in range(1, len(runs)):
            days_between = (runs[i]['start_date'] - runs[i-1]['end_date']).days
            runs[i]['days_since_last'] = days_between
        
        if runs:
            runs[0]['days_since_last'] = None
        
        return runs, hist
        
    except Exception as e:
        return None, str(e)

print("✅ find_runs() function ready")

✅ find_runs() function ready


In [4]:
def analyze_serial_runner(ticker, runs):
    """
    Analyze if a stock qualifies as a serial runner.
    
    Serial Runner Criteria:
    - 3+ runs in last 6 months
    - Consistent cycle (rest periods don't vary wildly)
    - Predictable gain range
    """
    if not runs or len(runs) < 3:
        return None
    
    # Calculate stats
    avg_run_length = np.mean([r['length'] for r in runs])
    avg_gain = np.mean([r['gain'] for r in runs])
    
    # Rest period stats (days between runs)
    rest_periods = [r['days_since_last'] for r in runs[1:] if r['days_since_last']]
    
    if rest_periods:
        avg_rest = np.mean(rest_periods)
        std_rest = np.std(rest_periods)
        rest_consistency = 1 - (std_rest / avg_rest) if avg_rest > 0 else 0
    else:
        avg_rest = None
        rest_consistency = 0
    
    # Gain consistency
    gains = [r['gain'] for r in runs]
    gain_std = np.std(gains)
    gain_consistency = 1 - (gain_std / avg_gain) if avg_gain > 0 else 0
    
    # Overall consistency score (0-100)
    consistency_score = (rest_consistency * 0.5 + gain_consistency * 0.5) * 100
    consistency_score = max(0, min(100, consistency_score))
    
    # Consistency label
    if consistency_score >= 70:
        consistency_label = 'HIGH'
    elif consistency_score >= 50:
        consistency_label = 'MEDIUM'
    else:
        consistency_label = 'LOW'
    
    # Predict next Day 1
    if avg_rest and runs[-1].get('end_date'):
        last_run_end = runs[-1]['end_date']
        expected_next_day1 = last_run_end + timedelta(days=int(avg_rest))
    else:
        expected_next_day1 = None
    
    return {
        'ticker': ticker,
        'num_runs': len(runs),
        'avg_run_length': round(avg_run_length, 1),
        'avg_gain': round(avg_gain, 1),
        'avg_rest_days': round(avg_rest, 1) if avg_rest else None,
        'consistency_score': round(consistency_score, 1),
        'consistency_label': consistency_label,
        'last_run_end': runs[-1].get('end_date'),
        'currently_running': runs[-1].get('active', False),
        'expected_next_day1': expected_next_day1,
        'runs': runs
    }

print("✅ analyze_serial_runner() function ready")

✅ analyze_serial_runner() function ready


In [5]:
# SCAN ALL TICKERS FOR SERIAL RUNNERS
print("\n" + "="*60)
print("🔍 SCANNING FOR SERIAL RUNNERS")
print("="*60)
print("\nCriteria: 3+ runs in 6 months, each run 3+ days, 10%+ gain\n")

serial_runners = []
all_results = {}

for sector, tickers in UNIVERSE.items():
    print(f"\n📊 {sector}:")
    print("-" * 40)
    
    for ticker in tickers:
        runs, hist = find_runs(ticker, min_run_days=3, min_gain_pct=10)
        
        if runs is None:
            print(f"  {ticker}: ❌ Error - {hist}")
            continue
        
        all_results[ticker] = {'runs': runs, 'hist': hist, 'sector': sector}
        
        if len(runs) >= 3:
            analysis = analyze_serial_runner(ticker, runs)
            if analysis:
                analysis['sector'] = sector
                serial_runners.append(analysis)
                print(f"  {ticker}: ✅ SERIAL RUNNER - {len(runs)} runs, avg {analysis['avg_gain']:.1f}% gain, "
                      f"{analysis['consistency_label']} consistency")
        elif len(runs) >= 1:
            print(f"  {ticker}: ⚠️ {len(runs)} runs (need 3+)")
        else:
            print(f"  {ticker}: ❌ No qualifying runs")

print(f"\n\n{'='*60}")
print(f"📊 SCAN COMPLETE: Found {len(serial_runners)} serial runners out of {len(ALL_TICKERS)} tickers")
print(f"{'='*60}")


🔍 SCANNING FOR SERIAL RUNNERS

Criteria: 3+ runs in 6 months, each run 3+ days, 10%+ gain


📊 Space:
----------------------------------------
  SIDU: ✅ SERIAL RUNNER - 7 runs, avg 40.9% gain, LOW consistency
  ASTS: ✅ SERIAL RUNNER - 11 runs, avg 28.8% gain, LOW consistency
  LUNR: ✅ SERIAL RUNNER - 8 runs, avg 24.6% gain, LOW consistency
  RKLB: ✅ SERIAL RUNNER - 9 runs, avg 26.8% gain, LOW consistency
  RDW: ✅ SERIAL RUNNER - 7 runs, avg 27.4% gain, LOW consistency

📊 Quantum:
----------------------------------------
  IONQ: ✅ SERIAL RUNNER - 9 runs, avg 21.5% gain, LOW consistency
  QBTS: ✅ SERIAL RUNNER - 10 runs, avg 32.1% gain, LOW consistency
  RGTI: ✅ SERIAL RUNNER - 11 runs, avg 28.5% gain, LOW consistency
  QUBT: ✅ SERIAL RUNNER - 7 runs, avg 24.1% gain, LOW consistency

📊 Semiconductor:
----------------------------------------
  NVTS: ✅ SERIAL RUNNER - 7 runs, avg 38.1% gain, LOW consistency
  WOLF: ⚠️ 2 runs (need 3+)
  ON: ⚠️ 2 runs (need 3+)
  AEHR: ✅ SERIAL RUNNER - 7 r

In [6]:
# SERIAL RUNNER DATABASE
if serial_runners:
    print("\n" + "="*80)
    print("🏆 SERIAL RUNNER DATABASE")
    print("="*80)
    
    # Sort by consistency score
    serial_runners_sorted = sorted(serial_runners, key=lambda x: x['consistency_score'], reverse=True)
    
    print(f"\n{'Ticker':<8} {'Sector':<12} {'Runs':<6} {'Avg Run':<10} {'Avg Gain':<10} {'Avg Rest':<10} {'Consistency':<12} {'Status'}")
    print("-" * 90)
    
    for sr in serial_runners_sorted:
        status = "🔥 RUNNING" if sr['currently_running'] else "💤 RESTING"
        rest_str = f"{sr['avg_rest_days']:.0f}d" if sr['avg_rest_days'] else "N/A"
        
        print(f"{sr['ticker']:<8} {sr['sector']:<12} {sr['num_runs']:<6} "
              f"{sr['avg_run_length']:.1f}d{'':<5} {sr['avg_gain']:+.1f}%{'':<4} "
              f"{rest_str:<10} {sr['consistency_label']:<12} {status}")
    
    print("\n" + "-" * 90)
    print(f"Total Serial Runners: {len(serial_runners)}")
    print(f"HIGH Consistency: {len([s for s in serial_runners if s['consistency_label'] == 'HIGH'])}")
    print(f"MEDIUM Consistency: {len([s for s in serial_runners if s['consistency_label'] == 'MEDIUM'])}")
    print(f"LOW Consistency: {len([s for s in serial_runners if s['consistency_label'] == 'LOW'])}")
else:
    print("\n❌ NO SERIAL RUNNERS FOUND - THESIS POTENTIALLY DEAD")


🏆 SERIAL RUNNER DATABASE

Ticker   Sector       Runs   Avg Run    Avg Gain   Avg Rest   Consistency  Status
------------------------------------------------------------------------------------------
CCJ      Nuclear      3      3.7d      +11.2%     15d        HIGH         🔥 RUNNING
MU       AI_Infra     7      5.9d      +22.1%     14d        MEDIUM       💤 RESTING
STX      AI_Infra     8      4.5d      +16.1%     14d        LOW          🔥 RUNNING
USAR     Rare_Earth   6      3.7d      +40.2%     27d        LOW          🔥 RUNNING
WDC      AI_Infra     7      4.9d      +19.1%     21d        LOW          🔥 RUNNING
QUBT     Quantum      7      3.3d      +24.1%     17d        LOW          🔥 RUNNING
LITE     AI_Infra     10     4.8d      +20.2%     12d        LOW          💤 RESTING
SMR      Nuclear      8      3.6d      +25.7%     21d        LOW          🔥 RUNNING
OKLO     Nuclear      9      3.8d      +26.1%     17d        LOW          🔥 RUNNING
LEU      Nuclear      6      3.2d      +18.8

In [7]:
# NEXT DAY 1 PREDICTIONS
if serial_runners:
    print("\n" + "="*80)
    print("🔮 NEXT DAY 1 PREDICTIONS")
    print("="*80)
    print("\nBased on average rest period after runs...\n")
    
    today = datetime.now().date()
    
    predictions = []
    for sr in serial_runners_sorted:
        if sr['currently_running']:
            # Predict when current run ends
            days_in_run = sr['runs'][-1]['length']
            expected_end = today + timedelta(days=int(sr['avg_run_length'] - days_in_run))
            expected_next = expected_end + timedelta(days=int(sr['avg_rest_days'] or 5))
            status = f"Running (Day {days_in_run})"
        elif sr['expected_next_day1']:
            expected_next = sr['expected_next_day1'].date() if hasattr(sr['expected_next_day1'], 'date') else sr['expected_next_day1']
            days_until = (expected_next - today).days if hasattr(expected_next, 'day') else 'Unknown'
            status = f"Resting ({days_until}d until Day 1)" if isinstance(days_until, int) else "Resting"
        else:
            expected_next = "Unknown"
            status = "Unknown"
        
        predictions.append({
            'ticker': sr['ticker'],
            'sector': sr['sector'],
            'status': status,
            'expected_next': expected_next,
            'consistency': sr['consistency_label']
        })
    
    print(f"{'Ticker':<8} {'Sector':<12} {'Status':<25} {'Next Day 1':<15} {'Consistency'}")
    print("-" * 75)
    
    for p in predictions:
        next_str = str(p['expected_next'])[:10] if p['expected_next'] != "Unknown" else "Unknown"
        print(f"{p['ticker']:<8} {p['sector']:<12} {p['status']:<25} {next_str:<15} {p['consistency']}")


🔮 NEXT DAY 1 PREDICTIONS

Based on average rest period after runs...

Ticker   Sector       Status                    Next Day 1      Consistency
---------------------------------------------------------------------------
CCJ      Nuclear      Running (Day 4)           2026-01-21      HIGH
MU       AI_Infra     Resting (-2d until Day 1) 2026-01-04      MEDIUM
STX      AI_Infra     Running (Day 3)           2026-01-20      LOW
USAR     Rare_Earth   Running (Day 3)           2026-02-02      LOW
WDC      AI_Infra     Running (Day 3)           2026-01-27      LOW
QUBT     Quantum      Running (Day 3)           2026-01-22      LOW
LITE     AI_Infra     Resting (-4d until Day 1) 2026-01-02      LOW
SMR      Nuclear      Running (Day 3)           2026-01-27      LOW
OKLO     Nuclear      Running (Day 4)           2026-01-23      LOW
LEU      Nuclear      Running (Day 3)           2026-02-06      LOW
UUUU     Nuclear      Resting (15d until Day 1) 2026-01-21      LOW
QBTS     Quantum      Run

In [8]:
# DETAILED RUN HISTORY FOR TOP SERIAL RUNNERS
if serial_runners:
    print("\n" + "="*80)
    print("📜 DETAILED RUN HISTORY (Top 5 by Consistency)")
    print("="*80)
    
    for sr in serial_runners_sorted[:5]:
        print(f"\n{'='*60}")
        print(f"🏃 {sr['ticker']} ({sr['sector']})")
        print(f"{'='*60}")
        print(f"Runs: {sr['num_runs']} | Avg Length: {sr['avg_run_length']}d | Avg Gain: {sr['avg_gain']:+.1f}%")
        print(f"Avg Rest: {sr['avg_rest_days']}d | Consistency: {sr['consistency_label']} ({sr['consistency_score']:.0f}/100)")
        print(f"\nRun History:")
        print(f"{'#':<4} {'Start':<12} {'End':<12} {'Days':<6} {'Gain':<10} {'Rest After'}")
        print("-" * 60)
        
        for i, run in enumerate(sr['runs'], 1):
            start = run['start_date'].strftime('%Y-%m-%d') if hasattr(run['start_date'], 'strftime') else str(run['start_date'])[:10]
            end = run['end_date'].strftime('%Y-%m-%d') if hasattr(run['end_date'], 'strftime') else str(run['end_date'])[:10]
            rest = f"{run.get('days_since_last', 'N/A')}d" if run.get('days_since_last') else "N/A"
            active = " (ACTIVE)" if run.get('active') else ""
            
            print(f"{i:<4} {start:<12} {end:<12} {run['length']:<6} {run['gain']:+.1f}%{'':<4} {rest}{active}")


📜 DETAILED RUN HISTORY (Top 5 by Consistency)

🏃 CCJ (Nuclear)
Runs: 3 | Avg Length: 3.7d | Avg Gain: +11.2%
Avg Rest: 15.0d | Consistency: HIGH (73/100)

Run History:
#    Start        End          Days   Gain       Rest After
------------------------------------------------------------
1    2025-11-24   2025-11-26   3      +11.8%     N/A
2    2025-12-18   2025-12-23   4      +10.1%     22d
3    2025-12-31   2026-01-06   4      +11.7%     8d (ACTIVE)

🏃 MU (AI_Infra)
Runs: 7 | Avg Length: 5.9d | Avg Gain: +22.1%
Avg Rest: 13.7d | Consistency: MEDIUM (53/100)

Run History:
#    Start        End          Days   Gain       Rest After
------------------------------------------------------------
1    2025-08-07   2025-08-12   4      +17.4%     N/A
2    2025-09-03   2025-09-18   12     +42.5%     22d
3    2025-09-26   2025-10-06   7      +21.8%     8d
4    2025-10-23   2025-10-29   5      +14.2%     17d
5    2025-11-21   2025-12-01   6      +19.4%     23d
6    2025-12-05   2025-12-10   4  

In [9]:
# THESIS VERDICT
print("\n" + "="*80)
print("🎯 THESIS VERDICT: SERIAL RUNNERS")
print("="*80)

if len(serial_runners) >= 5:
    print("\n✅ THESIS VALIDATED: Serial runners EXIST")
    print(f"   Found {len(serial_runners)} serial runners in {len(ALL_TICKERS)} tickers ({len(serial_runners)/len(ALL_TICKERS)*100:.0f}%)")
elif len(serial_runners) >= 3:
    print("\n⚠️ THESIS PARTIALLY VALIDATED: Some serial runners exist")
    print(f"   Found {len(serial_runners)} serial runners - borderline significant")
else:
    print("\n❌ THESIS KILLED: Not enough serial runners")
    print(f"   Only found {len(serial_runners)} - need 3+ for viable strategy")

if serial_runners:
    avg_consistency = np.mean([s['consistency_score'] for s in serial_runners])
    high_consistency = len([s for s in serial_runners if s['consistency_label'] == 'HIGH'])
    
    print(f"\n📊 Consistency Analysis:")
    print(f"   Average consistency score: {avg_consistency:.0f}/100")
    print(f"   HIGH consistency runners: {high_consistency}")
    
    if avg_consistency >= 60:
        print("   ✅ Cycles are PREDICTABLE enough to trade")
    else:
        print("   ⚠️ Cycles exist but timing is variable")
    
    avg_gain = np.mean([s['avg_gain'] for s in serial_runners])
    print(f"\n💰 Profit Potential:")
    print(f"   Average gain per run: {avg_gain:+.1f}%")
    
    if avg_gain >= 20:
        print("   ✅ Gains are WORTH the effort (20%+ per run)")
    elif avg_gain >= 15:
        print("   ⚠️ Gains are decent (15-20% per run)")
    else:
        print("   ❌ Gains may not justify the effort (<15% per run)")

print("\n" + "="*80)
print("NEXT: Run Notebook 2 (Day 0 Detection) to find entry signals")
print("="*80)


🎯 THESIS VERDICT: SERIAL RUNNERS

✅ THESIS VALIDATED: Serial runners EXIST
   Found 24 serial runners in 26 tickers (92%)

📊 Consistency Analysis:
   Average consistency score: 37/100
   HIGH consistency runners: 1
   ⚠️ Cycles exist but timing is variable

💰 Profit Potential:
   Average gain per run: +25.9%
   ✅ Gains are WORTH the effort (20%+ per run)

NEXT: Run Notebook 2 (Day 0 Detection) to find entry signals
